In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Llama 3 is up to 8k
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

csv_file_path = './dataset.csv'
df = pd.read_csv(csv_file_path)
df=df.dropna()
df=df.iloc[1:,[2,5]] # for singlish 
# df=df.iloc[1:,[0,3]] # for english
df

In [ ]:
prompt = """Below is an instruction that describes a task in singlish(romanized sinhala), paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
EOS_TOKEN = "<|endoftext|>" 

In [ ]:
data_dict = {
    "instruction": df.iloc[:, 0].tolist(),
    "input": [""] * len(df),
    "output": df.iloc[:, 1].tolist()
}
dataset = Dataset.from_dict(data_dict)


In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "What documents do I need to open a savings account?.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:

trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [2]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "maṭa masakaṭa kaḷa hæki ganudenu pramāṇayē sīmāvak tibēda?", # instruction
        "", # input
        "", # output 
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
#iturum giṇumē vargaya anuva obaṭa masakaṭa sidu kaḷa hæki ganudenu gaṇanē sīmāvan tibiya hæka.
# niścita ganudenu sīmāvan san̆dahā karuṇākara apagē giṇum viśēṣāṁga parīkṣā karanna.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
<|begin_of_text|>Below is an instruction that describes a task in singlish(romanized sinhala), paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
maṭa masakaṭa kaḷa hæki ganudenu pramāṇayē sīmāvak tibēda?

### Input:


### Response:
oba, sādhāranayen ganudenu pramāṇayē sīmāvak tīmī kandē. tīmī, vāhāruvā, vāhāruvāna kārṣana, vāhāruvāna kārṣana karṇa, vāhāruvāna kārṣana karṇa karṇa, vāhāruvāna kārṣana karṇa karṇa karṇa


In [ ]:
model.save_pretrained_merged("model3", tokenizer, save_method = "merged_16bit",)

In [ ]:
# add hugging face token ....removed my token for safety purposes.
model.push_to_hub_merged("Pavan04/llama-on-our-dataset-si", tokenizer, save_method = "merged_16bit", token = "")
